In [ ]:
from scapy.all import *

In [ ]:
packets = rdpcap('test.pcap')

In [ ]:
len(packets)

In [ ]:
packets[2].show()

In [ ]:
hexdump(packets[2])

In [ ]:
ls(packets[2])

In [ ]:
packets[2].summary()

In [ ]:
packets[2].lastlayer()

In [ ]:
packets[2].layers()

In [ ]:
packets[2].haslayer(TCP)

In [ ]:
packets[2].getlayer(TCP)

In [ ]:
packets[2].getlayer(TCP).payload

In [ ]:
bytes(packets[2].getlayer(TCP).payload)

In [ ]:
len(packets[2].getlayer(TCP).payload)

In [ ]:
packets[1].getlayer(Raw)

In [ ]:
packets[1].getlayer(Padding)

In [ ]:
packets[2].getlayer(TCP).fields

In [ ]:
packets[2].getlayer(Ether).fields

In [ ]:
# get all tcp packet sent to port 49600, exclude SYC packet and none data packet

dst_port = 49600

tcp_packets = list()

for packet in packets:
    if not packet.haslayer(TCP):
        continue
        
    if packet.getlayer(TCP).fields['dport'] != dst_port:
        continue
        
    if packet.getlayer(TCP).fields['flags'] & 0x02 == 0x02: # exclude SYN packet
        continue
        
    if not packet.haslayer(Raw): # exclude none data packet
        continue
        
    tcp_packets.append(packet.getlayer(TCP))


In [ ]:
tcp_packets = sorted(tcp_packets, key = lambda x : x.fields['seq'])

In [ ]:
# check duplicate packet

for i in range(0, len(tcp_packets)-1):
    if tcp_packets[i].fields['seq'] == tcp_packets[i + 1].fields['seq']:
        print('duplicate packet, i = {0}'.format(i))

In [ ]:
# check lose packet

for i in range(0, len(tcp_packets)-1):
    next_seq = tcp_packets[i].fields['seq'] + len(tcp_packets[i].getlayer(Raw))
    if tcp_packets[i].haslayer(Padding):
        next_seq -= len(tcp_packets[i].getlayer(Padding))
        
    if next_seq != tcp_packets[i + 1].fields['seq']:
        print('lose packet, i = {0}'.format(i))